In [1]:
import re
import pandas as pd

In [2]:
df_data = pd.read_csv("data.csv", encoding="latin-1")
df_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [3]:
df_data.shape

(13169, 13)

In [4]:
df_kamus = pd.read_csv("new_kamusalay.csv", encoding="latin-1", header=None)
df_kamus.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [5]:
# Removing the unnecessary columns.
df_data = df_data[['Tweet','HS','Abusive']]

df_data.head(10)

,Tweet,HS,Abusive
0,- disaat semua cowok berusaha melacak perhatia...,1,1
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1
5,USER Ya bani taplak dkk \xf0\x9f\x98\x84\xf0\x...,1,1
6,deklarasi pilkada 2018 aman dan anti hoax warg...,0,0
7,Gue baru aja kelar re-watch Aldnoah Zero!!! pa...,0,1
8,Nah admin belanja satu lagi port terbaik nak m...,0,0
9,USER Enak lg klo smbil ngewe',0,1


In [6]:
# LOWERCASING
def lowercase(text):
    return text.lower()

# DEFINING REGEX PATTERN
def regex_pattern(text):
    text = re.sub('\n',' ',text)
    text = re.sub('rt',' ',text)
    text = re.sub('user',' ',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    text = re.sub('  +', ' ', text)
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

# df_kamus AS A DICT
df_kamus_map = dict(zip(df_kamus[0], df_kamus[1]))
def normalize(text):
    return ' '.join([df_kamus_map[word] if word in df_kamus_map else word for word in text.split(' ')])

def preprocess_apply(text):
    text = lowercase(text)
    text = regex_pattern(text)
    text = normalize(text)
    return text

In [7]:
%%time
df_data['processed_text'] = df_data.Tweet.apply(preprocess_apply)

CPU times: total: 156 ms
Wall time: 232 ms


In [8]:
df_data.shape

(13169, 4)

In [9]:
df_data.head(20)

,Tweet,HS,Abusive,processed_text
0,- disaat semua cowok berusaha melacak perhatia...,1,1,di saat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,siapa yang telat memberi tau kamu edan sarap ...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,41 kadang aku berpikir kenapa aku tetap percay...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,aku itu aku dan ku tau matamu sipit tapi dili...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,kaum cebong kafir sudah kelihatan dongoknya d...
5,USER Ya bani taplak dkk \xf0\x9f\x98\x84\xf0\x...,1,1,ya bani taplak dan kawan kawan xf0 x9f x98 x8...
6,deklarasi pilkada 2018 aman dan anti hoax warg...,0,0,deklarasi pilihan kepala daerah 2018 aman dan ...
7,Gue baru aja kelar re-watch Aldnoah Zero!!! pa...,0,1,gue baru saja selesai re watch aldnoah zero pa...
8,Nah admin belanja satu lagi port terbaik nak m...,0,0,nah admin belanja satu lagi po terbaik nak mak...
9,USER Enak lg klo smbil ngewe',0,1,enak lagi kalau sambil ngewe
